In [55]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [56]:
# 'apple' : input_data => apple, output_data = pple!

In [57]:
input_str = 'apple'
label_str = 'pple!'
char_set = sorted(list(set(input_str+label_str)))
print(char_set)
char_set_size = len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


In [58]:
input_size = 5
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [59]:
char_to_index = dict((c,i) for i, c in enumerate(char_set))

# char_list = [c for c in char_set]
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [60]:
char_to_index.items()

dict_items([('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)])

In [61]:
char_to_index.values()

dict_values([0, 1, 2, 3, 4])

In [62]:
char_to_index.keys()

dict_keys(['!', 'a', 'e', 'l', 'p'])

In [63]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value]=key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [64]:
x_data = [char_to_index[c] for c in input_str]      # apple
y_data = [char_to_index[c] for c in label_str]      # pple!
x_data=[x_data]
y_data=[y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [65]:
x_one_hot = [np.eye(char_set_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [66]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [67]:
print(X.size(), Y.size())

torch.Size([1, 5, 5]) torch.Size([1, 5])


In [68]:
# nn.RNN(input_size, hidden_size, output_size)

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn=nn.RNN(input_size, hidden_size, batch_first=True)
        # batch_first : 입력 데이터의 배치의 차원의 위치를 설정
        # (배치크기, 시퀀스 길이, 특징 수)
        # 입력의 형태가 (32,10,50) : 배치의 크기 32, 시퀀스 길이 10, 특징의 수 50
        self.fc=nn.Linear(hidden_size, output_size, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [69]:
model = Net(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
output = model(X)
print(output)
print(output.size())

tensor([[[ 0.2148,  0.3756,  0.4254, -0.7129, -0.1679],
         [-0.2416,  0.0673,  0.4892, -0.5885, -0.4440],
         [-0.1719, -0.1547,  0.2520, -0.1486, -0.2568],
         [ 0.2114,  0.0415,  0.1348, -0.1999, -0.1248],
         [ 0.1521, -0.0465,  0.5170, -0.4732, -0.1279]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 5])


In [71]:
result = output.data.numpy().argmax(axis=2)
print(result)

[[2 2 2 0 2]]


In [72]:
for i in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = output.data.numpy().argmax(axis=2)
    str_result=''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, str_result, 'True: ', y_data)

0 loss:  1.7130171060562134 prediction:  [[2 2 2 0 2]] eee!e True:  [[4, 4, 3, 2, 0]]
1 loss:  1.4136368036270142 prediction:  [[2 2 3 0 2]] eel!e True:  [[4, 4, 3, 2, 0]]
2 loss:  1.1850433349609375 prediction:  [[4 3 3 2 0]] plle! True:  [[4, 4, 3, 2, 0]]
3 loss:  0.935154914855957 prediction:  [[4 4 3 3 0]] ppll! True:  [[4, 4, 3, 2, 0]]
4 loss:  0.7001248002052307 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
5 loss:  0.5150882601737976 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
6 loss:  0.35900866985321045 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
7 loss:  0.2491227686405182 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
8 loss:  0.16918864846229553 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
9 loss:  0.11367340385913849 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
10 loss:  0.07785370945930481 prediction:  [[4 4 3 2 0]] pple! True:  [[4, 4, 3, 2, 0]]
11 loss:  0.055260878056287766 prediction:  [[4 4 